<a href="https://colab.research.google.com/github/waynelee9511cloud/my-colab-notebooks/blob/main/Official_MedGemma_Training_1117.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ==================== CELL 1: 安裝套件 ====================
print('📦 開始安裝必要套件...')
!pip install -q transformers datasets accelerate bitsandbytes peft openpyxl scikit-learn matplotlib seaborn
print('✅ 套件安裝完成！')

📦 開始安裝必要套件...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 35.7 MB/s eta 0:00:00
✅ 套件安裝完成！


In [ ]:
# ==================== CELL 2: 導入函式庫 ====================
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, BitsAndBytesConfig
from datasets import Dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score, confusion_matrix
from sklearn.model_selection import KFold
from sklearn.utils import resample
from google.colab import files, drive
import os
import json
import numpy as np
import gc
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
print('✅ 所有函式庫導入完成！')

✅ 所有函式庫導入完成！


In [ ]:
# ==================== CELL 3: GPU 檢查 ====================
print('🔍 檢查 GPU 狀態...')
if not torch.cuda.is_available():
    raise RuntimeError('❌ 錯誤：需要 GPU 才能執行此程式！請確保已啟用 GPU。')

gpu_name = torch.cuda.get_device_name(0)
gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3

print(f'✅ GPU 已就緒！')
print(f'  GPU 型號: {gpu_name}')
print(f'  GPU 記憶體: {gpu_memory:.2f} GB')

# 清空 GPU 快取
torch.cuda.empty_cache()
gc.collect()
print('✅ GPU 快取已清空')

🔍 檢查 GPU 狀態...
✅ GPU 已就緒！
  GPU 型號: NVIDIA A100-SXM4-80GB
  GPU 記憶體: 79.32 GB
✅ GPU 快取已清空


In [ ]:
# ==================== CELL 4: 掛載 Google Drive ====================
print('📁 正在掛載 Google Drive...')
drive.mount('/content/drive')

# 檢查並創建輸出目錄
output_base_dir = '/content/drive/MyDrive/MedGemma_Training_Output'
os.makedirs(output_base_dir, exist_ok=True)

print('✅ Google Drive 掛載完成！')
print(f'  輸出目錄: {output_base_dir}')

📁 正在掛載 Google Drive...
Mounted at /content/drive
✅ Google Drive 掛載完成！
  輸出目錄: /content/drive/MyDrive/MedGemma_Training_Output


In [ ]:
# ==================== CELL 5: 上傳檔案 ====================
print('\n📤 請上傳您的訓練檔案')
print('  1. Excel 訓練資料檔案 (必須)')
print('  2. 台灣藥物商品名稱 txt 檔案（選填）')
print()

uploaded = files.upload()

excel_file = None
drug_names_file = None

for filename in uploaded.keys():
    if filename.endswith('.xlsx') or filename.endswith('.xls'):
        excel_file = filename
        print(f'✅ Excel 檔案: {excel_file}')
    elif filename.endswith('.txt'):
        drug_names_file = filename
        print(f'✅ 藥物名稱檔案: {drug_names_file}')

if not excel_file:
    raise ValueError('❌ 未找到 Excel 檔案！請上傳 .xlsx 或 .xls 格式的檔案。')

print('\n✅ 檔案上傳完成！')


📤 請上傳您的訓練檔案
  1. Excel 訓練資料檔案 (必須)
  2. 台灣藥物商品名稱 txt 檔案（選填）



Saving drug_names_list_for train.txt to drug_names_list_for train.txt
Saving Training data 1028.xlsx to Training data 1028.xlsx
✅ 藥物名稱檔案: drug_names_list_for train.txt
✅ Excel 檔案: Training data 1028.xlsx

✅ 檔案上傳完成！


In [ ]:
# ==================== CELL 6: 讀取台灣藥物名稱 ====================
taiwan_drug_names = set()

if drug_names_file:
    print(f'\n📋 正在讀取台灣藥物商品名稱...')
    try:
        with open(drug_names_file, 'r', encoding='utf-8') as f:
            for line in f:
                drug_name = line.strip()
                if drug_name:
                    taiwan_drug_names.add(drug_name)
        print(f'✅ 成功讀取 {len(taiwan_drug_names)} 個台灣藥物商品名稱')
        print(f'   範例: {list(taiwan_drug_names)[:5]}')
    except Exception as e:
        print(f'⚠️ 讀取藥物名稱檔案時發生錯誤: {e}')
        print(f'   將繼續執行，但不包含藥物名稱資料')
else:
    print('\n⚠️ 未提供台灣藥物名稱檔案，將僅使用 Excel 資料訓練')


📋 正在讀取台灣藥物商品名稱...
✅ 成功讀取 23455 個台灣藥物商品名稱
   範例: ['Thyrocure tablets 25 mcg', 'BEROGIN TABLETS 2.5MG (FENOTEROL) "HONTEN"', 'Solian Oral Solution 100mg/ml', 'VINKEBIR INJECTABLE SOLUTION¡]VINORELBINE 10MG/1ML/VIAL¡^', 'DOBUJECT 50MG/ML CONCENTRATE FOR PARENTERAL INFUSION']


In [ ]:
# ==================== CELL 7: 讀取 Excel 資料 ====================
print('\n📊 正在讀取 Excel 檔案...')

excel_data = pd.ExcelFile(excel_file)
sheet_names = excel_data.sheet_names
print(f'✅ 發現 {len(sheet_names)} 個工作表: {sheet_names}')

all_dataframes = []

for sheet_name in sheet_names:
    df_sheet = pd.read_excel(excel_data, sheet_name=sheet_name)

    required_columns = ['Term_To_Check', 'Correct_Output', 'category']
    missing_columns = [col for col in required_columns if col not in df_sheet.columns]

    if missing_columns:
        print(f'⚠️ 工作表 "{sheet_name}" 缺少欄位: {missing_columns}，跳過此工作表')
        continue

    total = len(df_sheet)
    correct = len(df_sheet[df_sheet['Correct_Output'] == 'No issues found.'])
    error = len(df_sheet[df_sheet['Correct_Output'] != 'No issues found.'])

    print(f'  📄 {sheet_name}: {total} 筆 (正確: {correct}, 錯誤: {error} / {error/total*100:.1f}%)')
    all_dataframes.append(df_sheet)

if not all_dataframes:
    raise ValueError('❌ 沒有找到任何有效的工作表！請確保 Excel 檔案包含必要的欄位。')

df = pd.concat(all_dataframes, ignore_index=True)

print(f'\n✅ 資料合併完成')
print(f'  總筆數: {len(df)}')
print(f'  正確術語: {len(df[df["Correct_Output"] == "No issues found."])} 筆')
print(f'  錯誤術語: {len(df[df["Correct_Output"] != "No issues found."])} 筆')


📊 正在讀取 Excel 檔案...
✅ 發現 3 個工作表: ['MH', 'CM', 'AE']
  📄 MH: 543 筆 (正確: 443, 錯誤: 100 / 18.4%)
  📄 CM: 530 筆 (正確: 384, 錯誤: 146 / 27.5%)
  📄 AE: 247 筆 (正確: 222, 錯誤: 25 / 10.1%)

✅ 資料合併完成
  總筆數: 1320
  正確術語: 1049 筆
  錯誤術語: 271 筆


In [ ]:
# ==================== CELL 8: 整合台灣藥物名稱 ====================
if taiwan_drug_names and len(taiwan_drug_names) > 0:
    print('\n' + '=' * 80)
    print('🇹🇼 整合台灣藥物商品名稱到訓練資料')
    print('=' * 80)

    drug_training_data = []
    for drug_name in taiwan_drug_names:
        drug_training_data.append({
            'Term_To_Check': drug_name,
            'Correct_Output': 'No issues found.',
            'category': 'CM'
        })

    df_drugs = pd.DataFrame(drug_training_data)
    original_len = len(df)
    df = pd.concat([df, df_drugs], ignore_index=True)

    print(f'\n✅ 成功整合 {len(df_drugs)} 個台灣藥物名稱')
    print(f'   原始資料: {original_len} 筆')
    print(f'   整合後: {len(df)} 筆')
    print('=' * 80)
else:
    print('\n⏭️ 跳過藥物名稱整合步驟')


🇹🇼 整合台灣藥物商品名稱到訓練資料

✅ 成功整合 23455 個台灣藥物名稱
   原始資料: 1320 筆
   整合後: 24775 筆


In [ ]:
# ==================== CELL 9: 資料平衡 ====================
print('\n' + '=' * 80)
print('📊 資料平衡處理')
print('=' * 80)

TARGET_ERROR_RATIO = 0.40
MAX_TOTAL_SAMPLES = 3500

current_error_count = len(df[df['Correct_Output'] != 'No issues found.'])
current_correct_count = len(df[df['Correct_Output'] == 'No issues found.'])
current_ratio = current_error_count / len(df)

print(f'\n📈 當前資料分布:')
print(f'  總筆數: {len(df)}')
print(f'  正確術語: {current_correct_count} ({current_correct_count/len(df)*100:.1f}%)')
print(f'  錯誤術語: {current_error_count} ({current_ratio*100:.1f}%)')
print(f'  目標錯誤比例: {TARGET_ERROR_RATIO*100:.1f}%')

if current_ratio < TARGET_ERROR_RATIO:
    print(f'\n⚙️ 錯誤比例 ({current_ratio*100:.1f}%) 低於目標 ({TARGET_ERROR_RATIO*100:.1f}%)，開始過採樣...')

    df_correct = df[df['Correct_Output'] == 'No issues found.'].copy()
    df_error = df[df['Correct_Output'] != 'No issues found.'].copy()

    # 計算需要的錯誤樣本數
    needed_error_count = int(TARGET_ERROR_RATIO * len(df_correct) / (1 - TARGET_ERROR_RATIO))
    potential_total = len(df_correct) + needed_error_count

    if potential_total > MAX_TOTAL_SAMPLES:
        print(f'  ⚠️ 預計總數 ({potential_total}) 超過上限 ({MAX_TOTAL_SAMPLES})，進行調整...')
        target_error_count = int(MAX_TOTAL_SAMPLES * TARGET_ERROR_RATIO)
        target_correct_count = MAX_TOTAL_SAMPLES - target_error_count

        # 如果正確樣本過多，進行下採樣
        if len(df_correct) > target_correct_count:
            print(f'  📉 對正確樣本進行下採樣: {len(df_correct)} -> {target_correct_count}')
            df_correct = resample(df_correct, replace=False, n_samples=target_correct_count, random_state=42)

        # 對錯誤樣本進行上採樣
        print(f'  📈 對錯誤樣本進行上採樣: {len(df_error)} -> {target_error_count}')
        df_error_upsampled = resample(df_error, replace=True, n_samples=target_error_count, random_state=42)
    else:
        print(f'  📈 對錯誤樣本進行上採樣: {len(df_error)} -> {needed_error_count}')
        df_error_upsampled = resample(df_error, replace=True, n_samples=needed_error_count, random_state=42)

    # 合併並打亂
    df = pd.concat([df_correct, df_error_upsampled], ignore_index=True)
    df = df.sample(frac=1, random_state=42).reset_index(drop=True)

    print(f'\n🎉 資料平衡完成！')
    print(f'  總筆數: {len(df)}')
    print(f'  正確術語: {len(df[df["Correct_Output"] == "No issues found."])} ({len(df[df["Correct_Output"] == "No issues found."])/len(df)*100:.1f}%)')
    print(f'  錯誤術語: {len(df[df["Correct_Output"] != "No issues found."])} ({len(df[df["Correct_Output"] != "No issues found."])/len(df)*100:.1f}%)')

    # 清理記憶體
    del df_correct, df_error, df_error_upsampled
    gc.collect()
else:
    print(f'\n✅ 資料比例已符合目標，無需調整')

print('=' * 80)


📊 資料平衡處理

📈 當前資料分布:
  總筆數: 24775
  正確術語: 24504 (98.9%)
  錯誤術語: 271 (1.1%)
  目標錯誤比例: 40.0%

⚙️ 錯誤比例 (1.1%) 低於目標 (40.0%)，開始過採樣...
  ⚠️ 預計總數 (40840) 超過上限 (3500)，進行調整...
  📉 對正確樣本進行下採樣: 24504 -> 2100
  📈 對錯誤樣本進行上採樣: 271 -> 1400

🎉 資料平衡完成！
  總筆數: 3500
  正確術語: 2100 (60.0%)
  錯誤術語: 1400 (40.0%)


In [ ]:
# ==================== CELL 10: 設定訓練參數 ====================
BASE_MODEL_ID = 'google/medgemma-4b-it'
LEARNING_RATE = 1e-4
NUM_EPOCHS = 3
BATCH_SIZE = 2
MAX_LENGTH = 384
GRADIENT_ACCUMULATION_STEPS = 8
N_SPLITS = 5

print(f'\n⚙️ 訓練參數設定')
print('=' * 80)
print(f'  模型: {BASE_MODEL_ID}')
print(f'  交叉驗證折數: {N_SPLITS}-fold')
print(f'  學習率: {LEARNING_RATE}')
print(f'  訓練輪數: {NUM_EPOCHS}')
print(f'  批次大小: {BATCH_SIZE}')
print(f'  最大序列長度: {MAX_LENGTH}')
print(f'  梯度累積步數: {GRADIENT_ACCUMULATION_STEPS}')
print(f'  有效批次大小: {BATCH_SIZE * GRADIENT_ACCUMULATION_STEPS}')
print('=' * 80)


⚙️ 訓練參數設定
  模型: google/medgemma-4b-it
  交叉驗證折數: 5-fold
  學習率: 0.0001
  訓練輪數: 3
  批次大小: 2
  最大序列長度: 384
  梯度累積步數: 8
  有效批次大小: 16


In [ ]:
# ==================== CELL 11: 準備訓練資料格式並初始化交叉驗證 ====================
print('\n🔄 準備訓練資料...')

def create_prompt(term):
    """創建 prompt"""
    return f'Please check if there is any error in this medical term: {term}'

def format_example(term, response, category):
    """格式化訓練範例"""
    prompt = create_prompt(term)
    formatted_text = f'<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n{response}<end_of_turn>'
    return {
        'text': formatted_text,
        'term': term,
        'response': response,
        'category': category
    }

# 準備所有訓練範例
all_examples = []
for idx, row in df.iterrows():
    example = format_example(
        term=row['Term_To_Check'],
        response=row['Correct_Output'],
        category=row['category']
    )
    all_examples.append(example)

print(f'✅ 準備了 {len(all_examples)} 個訓練範例')

# 初始化交叉驗證
print('\n' + '=' * 80)
print('🔄 初始化 5-Fold 交叉驗證')
print('=' * 80)

fold_results = {
    'accuracy': [],
    'precision': [],
    'recall': [],
    'f1': [],
    'confusion_matrices': [],
    'category_results': []
}

kfold = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
data_indices = list(range(len(all_examples)))

print(f'✅ 交叉驗證初始化完成')
print(f'  將資料分為 {N_SPLITS} 個 folds')
print(f'  每個 fold 約有 {len(all_examples)//N_SPLITS} 個測試樣本')
print('=' * 80)

print('\n⚠️ 重要提示：')
print('  接下來的 5 個 cells 將分別訓練 5 個 folds')
print('  請依序執行 Cell 12-16')
print('  每個 fold 訓練時間約 30-60 分鐘（取決於資料量）')


🔄 準備訓練資料...
✅ 準備了 3500 個訓練範例

🔄 初始化 5-Fold 交叉驗證
✅ 交叉驗證初始化完成
  將資料分為 5 個 folds
  每個 fold 約有 700 個測試樣本

⚠️ 重要提示：
  接下來的 5 個 cells 將分別訓練 5 個 folds
  請依序執行 Cell 12-16
  每個 fold 訓練時間約 30-60 分鐘（取決於資料量）


In [ ]:
# ==================== CELL 12: Fold 1 訓練 ====================
FOLD_IDX = 0  # Fold 1 (索引從 0 開始)

print('\n' + '=' * 80)
print(f'📊 Fold {FOLD_IDX + 1}/{N_SPLITS} 訓練')
print('=' * 80)

# 獲取此 fold 的訓練和測試索引
train_indices, test_indices = list(kfold.split(data_indices))[FOLD_IDX]

# 分割資料
train_examples = [all_examples[i] for i in train_indices]
test_examples = [all_examples[i] for i in test_indices]

print(f'  訓練集大小: {len(train_examples)}')
print(f'  測試集大小: {len(test_examples)}')

# 載入模型
print(f'\n🔧 Fold {FOLD_IDX + 1}: 載入模型...')
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL_ID,
    quantization_config=bnb_config,
    device_map='auto',
    trust_remote_code=True
)

print('✅ 模型載入完成')

# 配置 LoRA
print(f'\n🔧 Fold {FOLD_IDX + 1}: 配置 LoRA...')
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'],
    task_type='CAUSAL_LM',
    bias='none'
)

model = get_peft_model(model, lora_config)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())
print(f'✅ LoRA 配置完成')
print(f'  可訓練參數: {trainable_params:,} / {all_params:,} ({100 * trainable_params / all_params:.2f}%)')

# 準備訓練資料
print(f'\n🔄 Fold {FOLD_IDX + 1}: 準備訓練資料...')

def tokenize_function(examples):
    texts = examples['text'] if isinstance(examples['text'], list) else [examples['text']]
    tokenized = tokenizer(texts, truncation=True, padding='max_length', max_length=MAX_LENGTH, return_tensors=None)
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized

train_dataset_dict = {
    'text': [ex['text'] for ex in train_examples],
    'term': [ex['term'] for ex in train_examples],
    'response': [ex['response'] for ex in train_examples],
    'category': [ex['category'] for ex in train_examples]
}

test_dataset_dict = {
    'text': [ex['text'] for ex in test_examples],
    'term': [ex['term'] for ex in test_examples],
    'response': [ex['response'] for ex in test_examples],
    'category': [ex['category'] for ex in test_examples]
}

train_dataset = Dataset.from_dict(train_dataset_dict)
test_dataset = Dataset.from_dict(test_dataset_dict)

train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['text', 'term', 'response', 'category'], batch_size=100)
test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=['text', 'term', 'response', 'category'], batch_size=100)

print('✅ 資料準備完成')

# 清理記憶體
torch.cuda.empty_cache()
gc.collect()

# 設定訓練參數
output_dir = f'{output_base_dir}/fold_{FOLD_IDX + 1}'
os.makedirs(output_dir, exist_ok=True)

training_args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    learning_rate=LEARNING_RATE,
    lr_scheduler_type='cosine',
    warmup_ratio=0.1,
    eval_strategy='steps',
    eval_steps=100,
    save_strategy='steps',
    save_steps=100,
    save_total_limit=1,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    logging_steps=20,
    logging_dir=f'{output_dir}/logs',
    push_to_hub=False,
    report_to='none',
    fp16=True,
    remove_unused_columns=False,
    seed=42 + FOLD_IDX,
    data_seed=42 + FOLD_IDX,
    optim='paged_adamw_8bit',
    weight_decay=0.01,
    max_grad_norm=1.0,
    gradient_checkpointing=True,
    dataloader_num_workers=0,
)

# 訓練模型
print(f'\n🚀 Fold {FOLD_IDX + 1}: 開始訓練...')
print(f'  輸出目錄: {output_dir}')

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

train_result = trainer.train()
print(f'\n✅ Fold {FOLD_IDX + 1} 訓練完成')

# 保存模型
trainer.save_model(f'{output_dir}/final_model')
tokenizer.save_pretrained(f'{output_dir}/final_model')
print(f'✅ 模型已保存至: {output_dir}/final_model')

# 評估模型
print(f'\n📊 Fold {FOLD_IDX + 1}: 評估測試集...')

model.eval()
predictions = []
ground_truths = []
categories = []

for i, example in enumerate(test_examples):
    if i % 50 == 0:
        print(f'  進度: {i}/{len(test_examples)}')

    term = example['term']
    true_response = example['response']
    category = example['category']

    prompt = create_prompt(term)
    formatted = f'<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n'

    inputs = tokenizer(formatted, return_tensors='pt').to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id,
            do_sample=False,
            use_cache=True
        )

    pred_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True).strip()

    # 判斷預測結果
    pred_label = 1 if ('incorrect' in pred_text.lower() or 'error' in pred_text.lower()) else 0
    true_label = 0 if true_response == 'No issues found.' else 1

    predictions.append(pred_label)
    ground_truths.append(true_label)
    categories.append(category)

print(f'  進度: {len(test_examples)}/{len(test_examples)} ✅')

# 計算指標
accuracy = accuracy_score(ground_truths, predictions)
precision = precision_score(ground_truths, predictions, zero_division=0)
recall = recall_score(ground_truths, predictions, zero_division=0)
f1 = f1_score(ground_truths, predictions, zero_division=0)
cm = confusion_matrix(ground_truths, predictions)

# 保存結果
fold_results['accuracy'].append(accuracy)
fold_results['precision'].append(precision)
fold_results['recall'].append(recall)
fold_results['f1'].append(f1)
fold_results['confusion_matrices'].append(cm)

print(f'\n📈 Fold {FOLD_IDX + 1} 評估結果:')
print(f'  Accuracy:  {accuracy:.4f}')
print(f'  Precision: {precision:.4f}')
print(f'  Recall:    {recall:.4f}')
print(f'  F1 Score:  {f1:.4f}')
print(f'\n  混淆矩陣:')
print(f'  {cm}')

# 各類別性能分析
category_metrics = {}
unique_categories = set(categories)

for cat in unique_categories:
    cat_indices = [i for i, c in enumerate(categories) if c == cat]
    cat_predictions = [predictions[i] for i in cat_indices]
    cat_ground_truths = [ground_truths[i] for i in cat_indices]

    if len(cat_indices) > 0:
        cat_accuracy = accuracy_score(cat_ground_truths, cat_predictions)
        cat_precision = precision_score(cat_ground_truths, cat_predictions, zero_division=0)
        cat_recall = recall_score(cat_ground_truths, cat_predictions, zero_division=0)
        cat_f1 = f1_score(cat_ground_truths, cat_predictions, zero_division=0)

        category_metrics[cat] = {
            'accuracy': cat_accuracy,
            'precision': cat_precision,
            'recall': cat_recall,
            'f1': cat_f1,
            'samples': len(cat_indices)
        }

        print(f'\n  類別 {cat}:')
        print(f'    樣本數: {len(cat_indices)}')
        print(f'    F1: {cat_f1:.4f}, Acc: {cat_accuracy:.4f}')

fold_results['category_results'].append(category_metrics)

# 清理記憶體
del model, trainer, train_dataset, test_dataset
torch.cuda.empty_cache()
gc.collect()

print(f'\n✅ Fold {FOLD_IDX + 1} 完成！')
print('=' * 80)


📊 Fold 1/5 訓練
  訓練集大小: 2800
  測試集大小: 700

🔧 Fold 1: 載入模型...


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

...

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

✅ 資料準備完成

🚀 Fold 1: 開始訓練...
  輸出目錄: /content/drive/MyDrive/MedGemma_Training_Output/fold_1


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
100,0.675400,0.672704
200,0.511900,0.606951
300,0.495900,0.582322
400,0.392500,0.601553
500,0.440100,0.605030



✅ Fold 1 訓練完成
✅ 模型已保存至: /content/drive/MyDrive/MedGemma_Training_Output/fold_1/final_model

📊 Fold 1: 評估測試集...
  進度: 0/700
  進度: 50/700
  進度: 100/700
  進度: 150/700
  進度: 200/700
  進度: 250/700
  進度: 300/700
  進度: 350/700
  進度: 400/700
  進度: 450/700
  進度: 500/700
  進度: 550/700
  進度: 600/700
  進度: 650/700
  進度: 700/700 ✅

📈 Fold 1 評估結果:
  Accuracy:  0.9986
  Precision: 0.9963
  Recall:    1.0000
  F1 Score:  0.9981

  混淆矩陣:
  [[431   1]
 [  0 268]]

  類別 MH:
    樣本數: 110
    F1: 1.0000, Acc: 1.0000

  類別 AE:
    樣本數: 28
    F1: 0.9804, Acc: 0.9643

  類別 CM:
    樣本數: 562
    F1: 1.0000, Acc: 1.0000

✅ Fold 1 完成！


In [ ]:
# ==================== CELL 13: Fold 2 訓練 ====================
# 此 cell 與 Cell 12 結構相同，僅 FOLD_IDX 不同
FOLD_IDX = 1  # Fold 2

print('\n' + '=' * 80)
print(f'📊 Fold {FOLD_IDX + 1}/{N_SPLITS} 訓練')
print('=' * 80)

train_indices, test_indices = list(kfold.split(data_indices))[FOLD_IDX]
train_examples = [all_examples[i] for i in train_indices]
test_examples = [all_examples[i] for i in test_indices]

print(f'  訓練集大小: {len(train_examples)}')
print(f'  測試集大小: {len(test_examples)}')

print(f'\n🔧 Fold {FOLD_IDX + 1}: 載入模型...')
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type='nf4', bnb_4bit_compute_dtype=torch.float16, bnb_4bit_use_double_quant=True)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_ID, quantization_config=bnb_config, device_map='auto', trust_remote_code=True)
print('✅ 模型載入完成')

print(f'\n🔧 Fold {FOLD_IDX + 1}: 配置 LoRA...')
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.1, target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'], task_type='CAUSAL_LM', bias='none')
model = get_peft_model(model, lora_config)
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
all_params = sum(p.numel() for p in model.parameters())
print(f'✅ LoRA 配置完成 - 可訓練參數: {trainable_params:,} / {all_params:,} ({100 * trainable_params / all_params:.2f}%)')

print(f'\n🔄 Fold {FOLD_IDX + 1}: 準備訓練資料...')
def tokenize_function(examples):
    texts = examples['text'] if isinstance(examples['text'], list) else [examples['text']]
    tokenized = tokenizer(texts, truncation=True, padding='max_length', max_length=MAX_LENGTH, return_tensors=None)
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized

train_dataset = Dataset.from_dict({'text': [ex['text'] for ex in train_examples], 'term': [ex['term'] for ex in train_examples], 'response': [ex['response'] for ex in train_examples], 'category': [ex['category'] for ex in train_examples]})
test_dataset = Dataset.from_dict({'text': [ex['text'] for ex in test_examples], 'term': [ex['term'] for ex in test_examples], 'response': [ex['response'] for ex in test_examples], 'category': [ex['category'] for ex in test_examples]})
train_dataset = train_dataset.map(tokenize_function, batched=True, remove_columns=['text', 'term', 'response', 'category'], batch_size=100)
test_dataset = test_dataset.map(tokenize_function, batched=True, remove_columns=['text', 'term', 'response', 'category'], batch_size=100)
print('✅ 資料準備完成')
torch.cuda.empty_cache()
gc.collect()

output_dir = f'{output_base_dir}/fold_{FOLD_IDX + 1}'
os.makedirs(output_dir, exist_ok=True)
training_args = TrainingArguments(output_dir=output_dir, num_train_epochs=NUM_EPOCHS, per_device_train_batch_size=BATCH_SIZE, per_device_eval_batch_size=BATCH_SIZE, gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS, learning_rate=LEARNING_RATE, lr_scheduler_type='cosine', warmup_ratio=0.1, eval_strategy='steps', eval_steps=100, save_strategy='steps', save_steps=100, save_total_limit=1, load_best_model_at_end=True, metric_for_best_model='eval_loss', greater_is_better=False, logging_steps=20, logging_dir=f'{output_dir}/logs', push_to_hub=False, report_to='none', fp16=True, remove_unused_columns=False, seed=42 + FOLD_IDX, data_seed=42 + FOLD_IDX, optim='paged_adamw_8bit', weight_decay=0.01, max_grad_norm=1.0, gradient_checkpointing=True, dataloader_num_workers=0)

print(f'\n🚀 Fold {FOLD_IDX + 1}: 開始訓練...')
trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset)
train_result = trainer.train()
print(f'\n✅ Fold {FOLD_IDX + 1} 訓練完成')
trainer.save_model(f'{output_dir}/final_model')
tokenizer.save_pretrained(f'{output_dir}/final_model')

print(f'\n📊 Fold {FOLD_IDX + 1}: 評估測試集...')
model.eval()
predictions, ground_truths, categories = [], [], []
for i, example in enumerate(test_examples):
    if i % 50 == 0:
        print(f'  進度: {i}/{len(test_examples)}')
    term, true_response, category = example['term'], example['response'], example['category']
    prompt = create_prompt(term)
    formatted = f'<start_of_turn>user\n{prompt}<end_of_turn>\n<start_of_turn>model\n'
    inputs = tokenizer(formatted, return_tensors='pt').to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id, do_sample=False, use_cache=True)
    pred_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True).strip()
    pred_label = 1 if ('incorrect' in pred_text.lower() or 'error' in pred_text.lower()) else 0
    true_label = 0 if true_response == 'No issues found.' else 1
    predictions.append(pred_label)
    ground_truths.append(true_label)
    categories.append(category)

accuracy = accuracy_score(ground_truths, predictions)
precision = precision_score(ground_truths, predictions, zero_division=0)
recall = recall_score(ground_truths, predictions, zero_division=0)
f1 = f1_score(ground_truths, predictions, zero_division=0)
cm = confusion_matrix(ground_truths, predictions)
fold_results['accuracy'].append(accuracy)
fold_results['precision'].append(precision)
fold_results['recall'].append(recall)
fold_results['f1'].append(f1)
fold_results['confusion_matrices'].append(cm)

print(f'\n📈 Fold {FOLD_IDX + 1} 評估結果: Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')

category_metrics = {}
for cat in set(categories):
    cat_indices = [i for i, c in enumerate(categories) if c == cat]
    if len(cat_indices) > 0:
        cat_preds = [predictions[i] for i in cat_indices]
        cat_truths = [ground_truths[i] for i in cat_indices]
        category_metrics[cat] = {'accuracy': accuracy_score(cat_truths, cat_preds), 'precision': precision_score(cat_truths, cat_preds, zero_division=0), 'recall': recall_score(cat_truths, cat_preds, zero_division=0), 'f1': f1_score(cat_truths, cat_preds, zero_division=0), 'samples': len(cat_indices)}
fold_results['category_results'].append(category_metrics)

del model, trainer, train_dataset, test_dataset
torch.cuda.empty_cache()
gc.collect()
print(f'\n✅ Fold {FOLD_IDX + 1} 完成！')
print('=' * 80)


📊 Fold 2/5 訓練
  訓練集大小: 2800
  測試集大小: 700

🔧 Fold 2: 載入模型...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ 模型載入完成

🔧 Fold 2: 配置 LoRA...
✅ LoRA 配置完成 - 可訓練參數: 32,788,480 / 2,523,011,440 (1.30%)

🔄 Fold 2: 準備訓練資料...


Map:   0%|          | 0/2800 [00:00<?, ? examples/s]

Map:   0%|          | 0/700 [00:00<?, ? examples/s]

✅ 資料準備完成

🚀 Fold 2: 開始訓練...


Step,Training Loss,Validation Loss
100,0.665900,0.683623
200,0.515600,0.619077
300,0.514800,0.604149
400,0.394700,0.621165
500,0.394200,0.621799



✅ Fold 2 訓練完成

📊 Fold 2: 評估測試集...
  進度: 0/700
  進度: 50/700
  進度: 100/700
  進度: 150/700
  進度: 200/700
  進度: 250/700


In [ ]:
# ==================== CELL 14: Fold 3 訓練 ====================
FOLD_IDX = 2  # Fold 3

print('\n' + '=' * 80)
print(f'📊 Fold {FOLD_IDX + 1}/{N_SPLITS} 訓練')
print('=' * 80)

train_indices, test_indices = list(kfold.split(data_indices))[FOLD_IDX]
train_examples = [all_examples[i] for i in train_indices]
test_examples = [all_examples[i] for i in test_indices]
print(f'  訓練集大小: {len(train_examples)}, 測試集大小: {len(test_examples)}')

print(f'\n🔧 Fold {FOLD_IDX + 1}: 載入模型...')
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type='nf4', bnb_4bit_compute_dtype=torch.float16, bnb_4bit_use_double_quant=True)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_ID, quantization_config=bnb_config, device_map='auto', trust_remote_code=True)
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.1, target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'], task_type='CAUSAL_LM', bias='none')
model = get_peft_model(model, lora_config)
print('✅ 模型載入與 LoRA 配置完成')

def tokenize_function(examples):
    texts = examples['text'] if isinstance(examples['text'], list) else [examples['text']]
    tokenized = tokenizer(texts, truncation=True, padding='max_length', max_length=MAX_LENGTH, return_tensors=None)
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized

train_dataset = Dataset.from_dict({'text': [ex['text'] for ex in train_examples], 'term': [ex['term'] for ex in train_examples], 'response': [ex['response'] for ex in train_examples], 'category': [ex['category'] for ex in train_examples]}).map(tokenize_function, batched=True, remove_columns=['text', 'term', 'response', 'category'], batch_size=100)
test_dataset = Dataset.from_dict({'text': [ex['text'] for ex in test_examples], 'term': [ex['term'] for ex in test_examples], 'response': [ex['response'] for ex in test_examples], 'category': [ex['category'] for ex in test_examples]}).map(tokenize_function, batched=True, remove_columns=['text', 'term', 'response', 'category'], batch_size=100)
torch.cuda.empty_cache()
gc.collect()

output_dir = f'{output_base_dir}/fold_{FOLD_IDX + 1}'
os.makedirs(output_dir, exist_ok=True)
training_args = TrainingArguments(output_dir=output_dir, num_train_epochs=NUM_EPOCHS, per_device_train_batch_size=BATCH_SIZE, per_device_eval_batch_size=BATCH_SIZE, gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS, learning_rate=LEARNING_RATE, lr_scheduler_type='cosine', warmup_ratio=0.1, eval_strategy='steps', eval_steps=100, save_strategy='steps', save_steps=100, save_total_limit=1, load_best_model_at_end=True, metric_for_best_model='eval_loss', greater_is_better=False, logging_steps=20, logging_dir=f'{output_dir}/logs', push_to_hub=False, report_to='none', fp16=True, remove_unused_columns=False, seed=42 + FOLD_IDX, data_seed=42 + FOLD_IDX, optim='paged_adamw_8bit', weight_decay=0.01, max_grad_norm=1.0, gradient_checkpointing=True, dataloader_num_workers=0)

print(f'\n🚀 Fold {FOLD_IDX + 1}: 開始訓練...')
trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset)
trainer.train()
trainer.save_model(f'{output_dir}/final_model')
tokenizer.save_pretrained(f'{output_dir}/final_model')
print(f'✅ Fold {FOLD_IDX + 1} 訓練完成並保存')

print(f'\n📊 Fold {FOLD_IDX + 1}: 評估測試集...')
model.eval()
predictions, ground_truths, categories = [], [], []
for i, example in enumerate(test_examples):
    if i % 50 == 0:
        print(f'  進度: {i}/{len(test_examples)}')
    term, true_response, category = example['term'], example['response'], example['category']
    formatted = f'<start_of_turn>user\n{create_prompt(term)}<end_of_turn>\n<start_of_turn>model\n'
    inputs = tokenizer(formatted, return_tensors='pt').to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id, do_sample=False, use_cache=True)
    pred_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True).strip()
    predictions.append(1 if ('incorrect' in pred_text.lower() or 'error' in pred_text.lower()) else 0)
    ground_truths.append(0 if true_response == 'No issues found.' else 1)
    categories.append(category)

accuracy, precision, recall, f1 = accuracy_score(ground_truths, predictions), precision_score(ground_truths, predictions, zero_division=0), recall_score(ground_truths, predictions, zero_division=0), f1_score(ground_truths, predictions, zero_division=0)
cm = confusion_matrix(ground_truths, predictions)
fold_results['accuracy'].append(accuracy)
fold_results['precision'].append(precision)
fold_results['recall'].append(recall)
fold_results['f1'].append(f1)
fold_results['confusion_matrices'].append(cm)
print(f'\n📈 Fold {FOLD_IDX + 1} 結果: Acc={accuracy:.4f}, Prec={precision:.4f}, Rec={recall:.4f}, F1={f1:.4f}')

category_metrics = {}
for cat in set(categories):
    cat_indices = [i for i, c in enumerate(categories) if c == cat]
    if cat_indices:
        cat_preds, cat_truths = [predictions[i] for i in cat_indices], [ground_truths[i] for i in cat_indices]
        category_metrics[cat] = {'accuracy': accuracy_score(cat_truths, cat_preds), 'precision': precision_score(cat_truths, cat_preds, zero_division=0), 'recall': recall_score(cat_truths, cat_preds, zero_division=0), 'f1': f1_score(cat_truths, cat_preds, zero_division=0), 'samples': len(cat_indices)}
fold_results['category_results'].append(category_metrics)

del model, trainer, train_dataset, test_dataset
torch.cuda.empty_cache()
gc.collect()
print(f'✅ Fold {FOLD_IDX + 1} 完成！\n' + '=' * 80)

In [ ]:
# ==================== CELL 15: Fold 4 訓練 ====================
FOLD_IDX = 3  # Fold 4

print('\n' + '=' * 80)
print(f'📊 Fold {FOLD_IDX + 1}/{N_SPLITS} 訓練')
print('=' * 80)

train_indices, test_indices = list(kfold.split(data_indices))[FOLD_IDX]
train_examples = [all_examples[i] for i in train_indices]
test_examples = [all_examples[i] for i in test_indices]
print(f'  訓練集大小: {len(train_examples)}, 測試集大小: {len(test_examples)}')

print(f'\n🔧 Fold {FOLD_IDX + 1}: 載入模型...')
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type='nf4', bnb_4bit_compute_dtype=torch.float16, bnb_4bit_use_double_quant=True)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_ID, quantization_config=bnb_config, device_map='auto', trust_remote_code=True)
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.1, target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'], task_type='CAUSAL_LM', bias='none')
model = get_peft_model(model, lora_config)
print('✅ 模型載入與 LoRA 配置完成')

def tokenize_function(examples):
    texts = examples['text'] if isinstance(examples['text'], list) else [examples['text']]
    tokenized = tokenizer(texts, truncation=True, padding='max_length', max_length=MAX_LENGTH, return_tensors=None)
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized

train_dataset = Dataset.from_dict({'text': [ex['text'] for ex in train_examples], 'term': [ex['term'] for ex in train_examples], 'response': [ex['response'] for ex in train_examples], 'category': [ex['category'] for ex in train_examples]}).map(tokenize_function, batched=True, remove_columns=['text', 'term', 'response', 'category'], batch_size=100)
test_dataset = Dataset.from_dict({'text': [ex['text'] for ex in test_examples], 'term': [ex['term'] for ex in test_examples], 'response': [ex['response'] for ex in test_examples], 'category': [ex['category'] for ex in test_examples]}).map(tokenize_function, batched=True, remove_columns=['text', 'term', 'response', 'category'], batch_size=100)
torch.cuda.empty_cache()
gc.collect()

output_dir = f'{output_base_dir}/fold_{FOLD_IDX + 1}'
os.makedirs(output_dir, exist_ok=True)
training_args = TrainingArguments(output_dir=output_dir, num_train_epochs=NUM_EPOCHS, per_device_train_batch_size=BATCH_SIZE, per_device_eval_batch_size=BATCH_SIZE, gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS, learning_rate=LEARNING_RATE, lr_scheduler_type='cosine', warmup_ratio=0.1, eval_strategy='steps', eval_steps=100, save_strategy='steps', save_steps=100, save_total_limit=1, load_best_model_at_end=True, metric_for_best_model='eval_loss', greater_is_better=False, logging_steps=20, logging_dir=f'{output_dir}/logs', push_to_hub=False, report_to='none', fp16=True, remove_unused_columns=False, seed=42 + FOLD_IDX, data_seed=42 + FOLD_IDX, optim='paged_adamw_8bit', weight_decay=0.01, max_grad_norm=1.0, gradient_checkpointing=True, dataloader_num_workers=0)

print(f'\n🚀 Fold {FOLD_IDX + 1}: 開始訓練...')
trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset)
trainer.train()
trainer.save_model(f'{output_dir}/final_model')
tokenizer.save_pretrained(f'{output_dir}/final_model')
print(f'✅ Fold {FOLD_IDX + 1} 訓練完成並保存')

print(f'\n📊 Fold {FOLD_IDX + 1}: 評估測試集...')
model.eval()
predictions, ground_truths, categories = [], [], []
for i, example in enumerate(test_examples):
    if i % 50 == 0:
        print(f'  進度: {i}/{len(test_examples)}')
    term, true_response, category = example['term'], example['response'], example['category']
    formatted = f'<start_of_turn>user\n{create_prompt(term)}<end_of_turn>\n<start_of_turn>model\n'
    inputs = tokenizer(formatted, return_tensors='pt').to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id, do_sample=False, use_cache=True)
    pred_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True).strip()
    predictions.append(1 if ('incorrect' in pred_text.lower() or 'error' in pred_text.lower()) else 0)
    ground_truths.append(0 if true_response == 'No issues found.' else 1)
    categories.append(category)

accuracy, precision, recall, f1 = accuracy_score(ground_truths, predictions), precision_score(ground_truths, predictions, zero_division=0), recall_score(ground_truths, predictions, zero_division=0), f1_score(ground_truths, predictions, zero_division=0)
cm = confusion_matrix(ground_truths, predictions)
fold_results['accuracy'].append(accuracy)
fold_results['precision'].append(precision)
fold_results['recall'].append(recall)
fold_results['f1'].append(f1)
fold_results['confusion_matrices'].append(cm)
print(f'\n📈 Fold {FOLD_IDX + 1} 結果: Acc={accuracy:.4f}, Prec={precision:.4f}, Rec={recall:.4f}, F1={f1:.4f}')

category_metrics = {}
for cat in set(categories):
    cat_indices = [i for i, c in enumerate(categories) if c == cat]
    if cat_indices:
        cat_preds, cat_truths = [predictions[i] for i in cat_indices], [ground_truths[i] for i in cat_indices]
        category_metrics[cat] = {'accuracy': accuracy_score(cat_truths, cat_preds), 'precision': precision_score(cat_truths, cat_preds, zero_division=0), 'recall': recall_score(cat_truths, cat_preds, zero_division=0), 'f1': f1_score(cat_truths, cat_preds, zero_division=0), 'samples': len(cat_indices)}
fold_results['category_results'].append(category_metrics)

del model, trainer, train_dataset, test_dataset
torch.cuda.empty_cache()
gc.collect()
print(f'✅ Fold {FOLD_IDX + 1} 完成！\n' + '=' * 80)

In [ ]:
# ==================== CELL 16: Fold 5 訓練 ====================
FOLD_IDX = 4  # Fold 5

print('\n' + '=' * 80)
print(f'📊 Fold {FOLD_IDX + 1}/{N_SPLITS} 訓練 (最後一個 Fold)')
print('=' * 80)

train_indices, test_indices = list(kfold.split(data_indices))[FOLD_IDX]
train_examples = [all_examples[i] for i in train_indices]
test_examples = [all_examples[i] for i in test_indices]
print(f'  訓練集大小: {len(train_examples)}, 測試集大小: {len(test_examples)}')

print(f'\n🔧 Fold {FOLD_IDX + 1}: 載入模型...')
bnb_config = BitsAndBytesConfig(load_in_4bit=True, bnb_4bit_quant_type='nf4', bnb_4bit_compute_dtype=torch.float16, bnb_4bit_use_double_quant=True)
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_ID)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'
model = AutoModelForCausalLM.from_pretrained(BASE_MODEL_ID, quantization_config=bnb_config, device_map='auto', trust_remote_code=True)
model = prepare_model_for_kbit_training(model)
lora_config = LoraConfig(r=16, lora_alpha=32, lora_dropout=0.1, target_modules=['q_proj', 'k_proj', 'v_proj', 'o_proj', 'gate_proj', 'up_proj', 'down_proj'], task_type='CAUSAL_LM', bias='none')
model = get_peft_model(model, lora_config)
print('✅ 模型載入與 LoRA 配置完成')

def tokenize_function(examples):
    texts = examples['text'] if isinstance(examples['text'], list) else [examples['text']]
    tokenized = tokenizer(texts, truncation=True, padding='max_length', max_length=MAX_LENGTH, return_tensors=None)
    tokenized['labels'] = tokenized['input_ids'].copy()
    return tokenized

train_dataset = Dataset.from_dict({'text': [ex['text'] for ex in train_examples], 'term': [ex['term'] for ex in train_examples], 'response': [ex['response'] for ex in train_examples], 'category': [ex['category'] for ex in train_examples]}).map(tokenize_function, batched=True, remove_columns=['text', 'term', 'response', 'category'], batch_size=100)
test_dataset = Dataset.from_dict({'text': [ex['text'] for ex in test_examples], 'term': [ex['term'] for ex in test_examples], 'response': [ex['response'] for ex in test_examples], 'category': [ex['category'] for ex in test_examples]}).map(tokenize_function, batched=True, remove_columns=['text', 'term', 'response', 'category'], batch_size=100)
torch.cuda.empty_cache()
gc.collect()

output_dir = f'{output_base_dir}/fold_{FOLD_IDX + 1}'
os.makedirs(output_dir, exist_ok=True)
training_args = TrainingArguments(output_dir=output_dir, num_train_epochs=NUM_EPOCHS, per_device_train_batch_size=BATCH_SIZE, per_device_eval_batch_size=BATCH_SIZE, gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS, learning_rate=LEARNING_RATE, lr_scheduler_type='cosine', warmup_ratio=0.1, eval_strategy='steps', eval_steps=100, save_strategy='steps', save_steps=100, save_total_limit=1, load_best_model_at_end=True, metric_for_best_model='eval_loss', greater_is_better=False, logging_steps=20, logging_dir=f'{output_dir}/logs', push_to_hub=False, report_to='none', fp16=True, remove_unused_columns=False, seed=42 + FOLD_IDX, data_seed=42 + FOLD_IDX, optim='paged_adamw_8bit', weight_decay=0.01, max_grad_norm=1.0, gradient_checkpointing=True, dataloader_num_workers=0)

print(f'\n🚀 Fold {FOLD_IDX + 1}: 開始訓練...')
trainer = Trainer(model=model, args=training_args, train_dataset=train_dataset, eval_dataset=test_dataset)
trainer.train()
trainer.save_model(f'{output_dir}/final_model')
tokenizer.save_pretrained(f'{output_dir}/final_model')
print(f'✅ Fold {FOLD_IDX + 1} 訓練完成並保存')

print(f'\n📊 Fold {FOLD_IDX + 1}: 評估測試集...')
model.eval()
predictions, ground_truths, categories = [], [], []
for i, example in enumerate(test_examples):
    if i % 50 == 0:
        print(f'  進度: {i}/{len(test_examples)}')
    term, true_response, category = example['term'], example['response'], example['category']
    formatted = f'<start_of_turn>user\n{create_prompt(term)}<end_of_turn>\n<start_of_turn>model\n'
    inputs = tokenizer(formatted, return_tensors='pt').to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=100, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id, do_sample=False, use_cache=True)
    pred_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:], skip_special_tokens=True).strip()
    predictions.append(1 if ('incorrect' in pred_text.lower() or 'error' in pred_text.lower()) else 0)
    ground_truths.append(0 if true_response == 'No issues found.' else 1)
    categories.append(category)

accuracy, precision, recall, f1 = accuracy_score(ground_truths, predictions), precision_score(ground_truths, predictions, zero_division=0), recall_score(ground_truths, predictions, zero_division=0), f1_score(ground_truths, predictions, zero_division=0)
cm = confusion_matrix(ground_truths, predictions)
fold_results['accuracy'].append(accuracy)
fold_results['precision'].append(precision)
fold_results['recall'].append(recall)
fold_results['f1'].append(f1)
fold_results['confusion_matrices'].append(cm)
print(f'\n📈 Fold {FOLD_IDX + 1} 結果: Acc={accuracy:.4f}, Prec={precision:.4f}, Rec={recall:.4f}, F1={f1:.4f}')

category_metrics = {}
for cat in set(categories):
    cat_indices = [i for i, c in enumerate(categories) if c == cat]
    if cat_indices:
        cat_preds, cat_truths = [predictions[i] for i in cat_indices], [ground_truths[i] for i in cat_indices]
        category_metrics[cat] = {'accuracy': accuracy_score(cat_truths, cat_preds), 'precision': precision_score(cat_truths, cat_preds, zero_division=0), 'recall': recall_score(cat_truths, cat_preds, zero_division=0), 'f1': f1_score(cat_truths, cat_preds, zero_division=0), 'samples': len(cat_indices)}
fold_results['category_results'].append(category_metrics)

del model, trainer, train_dataset, test_dataset
torch.cuda.empty_cache()
gc.collect()
print(f'✅ Fold {FOLD_IDX + 1} 完成！')
print('=' * 80)
print('\n🎉 所有 5 個 Folds 訓練完成！請繼續執行後續的統計分析 cells。')

In [ ]:
# ==================== CELL 17: 統計分析 ====================
print('\n' + '=' * 80)
print('📊 交叉驗證統計分析')
print('=' * 80)

# 確保所有 5 個 folds 都已完成
if len(fold_results['f1']) != N_SPLITS:
    print(f'\n⚠️ 警告：目前只完成了 {len(fold_results["f1"])} 個 folds，需要完成 {N_SPLITS} 個！')
    print('請確保已執行所有 fold 訓練 cells (Cell 12-16)')
else:
    print(f'\n✅ 所有 {N_SPLITS} 個 folds 已完成')

metrics_stats = {}
for metric in ['accuracy', 'precision', 'recall', 'f1']:
    values = fold_results[metric]
    mean = np.mean(values)
    std = np.std(values, ddof=1)

    # 計算 95% 信賴區間
    confidence_level = 0.95
    degrees_freedom = len(values) - 1
    confidence_interval = stats.t.interval(confidence_level, degrees_freedom, mean, stats.sem(values))

    metrics_stats[metric] = {
        'mean': mean,
        'std': std,
        'ci_lower': confidence_interval[0],
        'ci_upper': confidence_interval[1],
        'values': values
    }

    print(f'\n{metric.upper()}:')
    print(f'  各 Fold 結果: {[f"{v:.4f}" for v in values]}')
    print(f'  平均值: {mean:.4f}')
    print(f'  標準差: {std:.4f}')
    print(f'  95% 信賴區間: [{confidence_interval[0]:.4f}, {confidence_interval[1]:.4f}]')

print('\n' + '=' * 80)

In [ ]:
# ==================== CELL 18: 混淆矩陣視覺化 ====================
print('\n📊 生成混淆矩陣視覺化...')

# 計算平均混淆矩陣
avg_cm = np.mean(fold_results['confusion_matrices'], axis=0)

# 創建圖表
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('5-Fold Cross Validation - Confusion Matrices', fontsize=16, fontweight='bold')

# 繪製每個 fold 的混淆矩陣
for idx in range(N_SPLITS):
    row = idx // 3
    col = idx % 3
    ax = axes[row, col]

    cm = fold_results['confusion_matrices'][idx]
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
                xticklabels=['Correct', 'Error'],
                yticklabels=['Correct', 'Error'],
                cbar_kws={'label': 'Count'})
    ax.set_title(f'Fold {idx + 1}\nF1={fold_results["f1"][idx]:.4f}', fontsize=12)
    ax.set_ylabel('True Label')
    ax.set_xlabel('Predicted Label')

# 繪製平均混淆矩陣
ax = axes[1, 2]
sns.heatmap(avg_cm, annot=True, fmt='.1f', cmap='Greens', ax=ax,
            xticklabels=['Correct', 'Error'],
            yticklabels=['Correct', 'Error'],
            cbar_kws={'label': 'Average Count'})
ax.set_title(f'Average Confusion Matrix\nF1={metrics_stats["f1"]["mean"]:.4f}±{metrics_stats["f1"]["std"]:.4f}',
             fontsize=12, fontweight='bold')
ax.set_ylabel('True Label')
ax.set_xlabel('Predicted Label')

plt.tight_layout()

# 保存圖表
output_path = f'{output_base_dir}/confusion_matrices_cv.png'
plt.savefig(output_path, dpi=300, bbox_inches='tight')
print(f'✅ 混淆矩陣已保存至: {output_path}')
plt.show()

print('\n混淆矩陣已顯示完成')

In [ ]:
# ==================== CELL 19: 性能指標視覺化 ====================
print('\n📊 生成性能指標視覺化...')

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('5-Fold Cross Validation - Performance Metrics', fontsize=16, fontweight='bold')

metrics_to_plot = ['accuracy', 'precision', 'recall', 'f1']
metric_names = ['Accuracy', 'Precision', 'Recall', 'F1 Score']

for idx, (metric, name) in enumerate(zip(metrics_to_plot, metric_names)):
    row = idx // 2
    col = idx % 2
    ax = axes[row, col]

    values = metrics_stats[metric]['values']
    mean = metrics_stats[metric]['mean']
    ci_lower = metrics_stats[metric]['ci_lower']
    ci_upper = metrics_stats[metric]['ci_upper']

    folds = list(range(1, N_SPLITS + 1))

    # 繪製折線圖
    ax.plot(folds, values, marker='o', linewidth=2, markersize=8,
            label='Fold Results', color='steelblue')

    # 繪製平均值線
    ax.axhline(y=mean, color='red', linestyle='--', linewidth=2,
               label=f'Mean: {mean:.4f}')

    # 繪製信賴區間
    ax.axhspan(ci_lower, ci_upper, alpha=0.2, color='red',
               label=f'95% CI')

    ax.set_xlabel('Fold', fontsize=12)
    ax.set_ylabel(name, fontsize=12)
    ax.set_title(f'{name}\nMean: {mean:.4f} ± {metrics_stats[metric]["std"]:.4f}',
                 fontsize=12, fontweight='bold')
    ax.set_xticks(folds)
    ax.grid(True, alpha=0.3)
    ax.legend(loc='best')

    # 設定 y 軸範圍
    y_min = max(0.0, min(values) - 0.05)
    y_max = min(1.0, max(values) + 0.05)
    ax.set_ylim([y_min, y_max])

plt.tight_layout()

# 保存圖表
output_path = f'{output_base_dir}/performance_metrics_cv.png'
plt.savefig(output_path, dpi=300, bbox_inches='tight')
print(f'✅ 性能指標圖已保存至: {output_path}')
plt.show()

print('\n性能指標視覺化已完成')

In [ ]:
# ==================== CELL 20: 生成詳細報告 ====================
print('\n📝 生成詳細報告...')

report_lines = []
report_lines.append('=' * 80)
report_lines.append('MedGemma 醫療術語校正模型 - 5-Fold 交叉驗證報告')
report_lines.append('=' * 80)
report_lines.append('')

# 基本資訊
report_lines.append('【基本資訊】')
report_lines.append(f'  模型: {BASE_MODEL_ID}')
report_lines.append(f'  訓練資料總數: {len(all_examples)}')
report_lines.append(f'  交叉驗證折數: {N_SPLITS}')
report_lines.append(f'  訓練輪數: {NUM_EPOCHS}')
report_lines.append(f'  學習率: {LEARNING_RATE}')
report_lines.append(f'  批次大小: {BATCH_SIZE}')
report_lines.append(f'  梯度累積步數: {GRADIENT_ACCUMULATION_STEPS}')
if taiwan_drug_names:
    report_lines.append(f'  台灣藥物名稱數量: {len(taiwan_drug_names)}')
report_lines.append('')

# 整體性能統計
report_lines.append('【整體性能統計】')
for metric, name in zip(['accuracy', 'precision', 'recall', 'f1'],
                       ['Accuracy', 'Precision', 'Recall', 'F1 Score']):
    stats_data = metrics_stats[metric]
    report_lines.append(f'\n{name}:')
    report_lines.append(f'  平均值: {stats_data["mean"]:.4f}')
    report_lines.append(f'  標準差: {stats_data["std"]:.4f}')
    report_lines.append(f'  95% 信賴區間: [{stats_data["ci_lower"]:.4f}, {stats_data["ci_upper"]:.4f}]')
    report_lines.append(f'  各 Fold 結果: {[f"{v:.4f}" for v in stats_data["values"]]}')

# 各 Fold 詳細結果
report_lines.append('\n' + '=' * 80)
report_lines.append('【各 Fold 詳細結果】')
for i in range(N_SPLITS):
    report_lines.append(f'\nFold {i+1}:')
    report_lines.append(f'  Accuracy:  {fold_results["accuracy"][i]:.4f}')
    report_lines.append(f'  Precision: {fold_results["precision"][i]:.4f}')
    report_lines.append(f'  Recall:    {fold_results["recall"][i]:.4f}')
    report_lines.append(f'  F1 Score:  {fold_results["f1"][i]:.4f}')
    report_lines.append(f'  混淆矩陣:\n  {fold_results["confusion_matrices"][i]}')

# 結論
report_lines.append('\n' + '=' * 80)
report_lines.append('【結論】')
report_lines.append(f'本次 5-Fold 交叉驗證訓練已完成。')
report_lines.append(f'平均 F1 Score: {metrics_stats["f1"]["mean"]:.4f} ± {metrics_stats["f1"]["std"]:.4f}')
report_lines.append(f'平均 Accuracy: {metrics_stats["accuracy"]["mean"]:.4f} ± {metrics_stats["accuracy"]["std"]:.4f}')
report_lines.append('')
report_lines.append('模型檔案保存位置:')
for i in range(N_SPLITS):
    report_lines.append(f'  Fold {i+1}: {output_base_dir}/fold_{i+1}/final_model')
report_lines.append('')
report_lines.append('視覺化圖表保存位置:')
report_lines.append(f'  混淆矩陣: {output_base_dir}/confusion_matrices_cv.png')
report_lines.append(f'  性能指標: {output_base_dir}/performance_metrics_cv.png')
report_lines.append('=' * 80)

# 生成報告文字
report_text = '\n'.join(report_lines)

# 保存報告
report_path = f'{output_base_dir}/cv_report.txt'
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(report_text)

print(f'✅ 詳細報告已保存至: {report_path}')
print('\n' + '=' * 80)
print('報告內容:')
print('=' * 80)
print(report_text)

print('\n' + '=' * 80)
print('🎉 5-Fold 交叉驗證完成！')
print('=' * 80)
print(f'\n📊 最終結果摘要:')
print(f'  F1 Score:  {metrics_stats["f1"]["mean"]:.4f} ± {metrics_stats["f1"]["std"]:.4f}')
print(f'  Accuracy:  {metrics_stats["accuracy"]["mean"]:.4f} ± {metrics_stats["accuracy"]["std"]:.4f}')
print(f'  Precision: {metrics_stats["precision"]["mean"]:.4f} ± {metrics_stats["precision"]["std"]:.4f}')
print(f'  Recall:    {metrics_stats["recall"]["mean"]:.4f} ± {metrics_stats["recall"]["std"]:.4f}')
print('\n所有結果已保存至 Google Drive')
print(f'輸出目錄: {output_base_dir}')
print('=' * 80)